# Data cleaning example

In [1]:
import pandas as pd
import numpy as np

flight.txt:

```
From_to, SerialNumber, Delays, Airline
paris_LONdon, 20020, 20 4 1, United Airlines.
MAdrid_Rome, *, "13", (British Airways. )
OSLO_PaRIS, 20040, , <Air France>
stockholm_berlin,*, 43 5, "Swiss Air"
```

In [2]:
df = pd.read_csv('flight.txt')
df

,From_to,SerialNumber,Delays,Airline
0,paris_LONdon,20020,20 4 1,United Airlines.
1,MAdrid_Rome,,"""13""",(British Airways. )
2,OSLO_newYork,,,<Air France>
3,stockholm_berlin,20050,43 5,"""Swiss Air"""


There are several problems of this read:
* There are spaces before of after the commas in the CSV file
* Upper- and lowercase issues
* SerialNumbers should be integers
* Delays should be lists
* Airline names needs to be cleaned up

For example

In [3]:
df.columns

Index(['From_to', ' SerialNumber', ' Delays', ' Airline'], dtype='object')

In [4]:
df[' SerialNumber'][0]

' 20020'

Use the option to fix the spaces in the CSV file

In [5]:
df = pd.read_csv('flight.txt', skipinitialspace=True)
df

,From_to,SerialNumber,Delays,Airline
0,paris_LONdon,20020.0,20 4 1,United Airlines.
1,MAdrid_Rome,NaN,13,(British Airways. )
2,OSLO_newYork,NaN,NaN,<Air France>
3,stockholm_berlin,20050.0,43 5,Swiss Air


In [6]:
df.columns

Index(['From_to', 'SerialNumber', 'Delays', 'Airline'], dtype='object')

In [7]:
df['SerialNumber'][0]

20020.0

In [8]:
df.dtypes['SerialNumber'] = np.float64

In [9]:
type(df['SerialNumber'][0])

numpy.float64

# Fixing the `SerialNumber` column

NaN prevents some computation from being performed. We can replace them with values if desired

In [10]:
df['SerialNumber'].fillna(-1)

0    20020.0
1       -1.0
2       -1.0
3    20050.0
Name: SerialNumber, dtype: float64

In [11]:
df['SerialNumber'].fillna(0).astype(np.int64)

0    20020
1        0
2        0
3    20050
Name: SerialNumber, dtype: int64

Interpolation of the NaN values

In [12]:
df['SerialNumber'].interpolate()

0    20020.0
1    20030.0
2    20040.0
3    20050.0
Name: SerialNumber, dtype: float64

In [13]:
df['SerialNumber'] = df['SerialNumber'].interpolate().astype(np.int64)
df

,From_to,SerialNumber,Delays,Airline
0,paris_LONdon,20020,20 4 1,United Airlines.
1,MAdrid_Rome,20030,13,(British Airways. )
2,OSLO_newYork,20040,NaN,<Air France>
3,stockholm_berlin,20050,43 5,Swiss Air


## Fixing the `From_to` column

In [14]:
df['From_to'].str.split('_')

0        [paris, LONdon]
1         [MAdrid, Rome]
2        [OSLO, newYork]
3    [stockholm, berlin]
Name: From_to, dtype: object

In [15]:
df['From_to'].str.split('_', expand=True)

,0,1
0,paris,LONdon
1,MAdrid,Rome
2,OSLO,newYork
3,stockholm,berlin


In [16]:
temp = df['From_to'].str.split('_', expand=True)
temp

,0,1
0,paris,LONdon
1,MAdrid,Rome
2,OSLO,newYork
3,stockholm,berlin


In [17]:
temp = df['From_to'].str.split('_', expand=True)
temp.columns = ['From', 'To']
temp

,From,To
0,paris,LONdon
1,MAdrid,Rome
2,OSLO,newYork
3,stockholm,berlin


In [18]:
temp2 = temp[ ['To', 'From']    ]
temp2

,To,From
0,LONdon,paris
1,Rome,MAdrid
2,newYork,OSLO
3,berlin,stockholm


In [19]:
df = df.join(temp)
df

,From_to,SerialNumber,Delays,Airline,From,To
0,paris_LONdon,20020,20 4 1,United Airlines.,paris,LONdon
1,MAdrid_Rome,20030,13,(British Airways. ),MAdrid,Rome
2,OSLO_newYork,20040,NaN,<Air France>,OSLO,newYork
3,stockholm_berlin,20050,43 5,Swiss Air,stockholm,berlin


Now we can delete the original "From_to" column:

In [20]:
df = df.drop('From_to', axis=1)
df

,SerialNumber,Delays,Airline,From,To
0,20020,20 4 1,United Airlines.,paris,LONdon
1,20030,13,(British Airways. ),MAdrid,Rome
2,20040,NaN,<Air France>,OSLO,newYork
3,20050,43 5,Swiss Air,stockholm,berlin


# Fixing the mixed upper/lower cases

In [21]:
df['From'].str.capitalize()

0        Paris
1       Madrid
2         Oslo
3    Stockholm
Name: From, dtype: object

In [22]:
df['From'] = df['From'].str.capitalize()
df['To'] = df['To'].str.capitalize()
df

,SerialNumber,Delays,Airline,From,To
0,20020,20 4 1,United Airlines.,Paris,London
1,20030,13,(British Airways. ),Madrid,Rome
2,20040,NaN,<Air France>,Oslo,Newyork
3,20050,43 5,Swiss Air,Stockholm,Berlin


# Fixing the Airline names

In [41]:
df['Airline']

0    United Airlines
1    British Airways
2         Air France
3          Swiss Air
Name: Airline, dtype: object

We want to remove the unwanted characters and spaces

In [42]:
df['Airline'].str.extract('([a-zA-Z\s]+)', expand=True)

,0
0,United Airlines
1,British Airways
2,Air France
3,Swiss Air


In [25]:
df['Airline'] = df['Airline'].str.extract('([a-zA-Z\s]+)', expand=True)
df

,SerialNumber,Delays,Airline,From,To
0,20020,20 4 1,United Airlines,Paris,London
1,20030,13,British Airways,Madrid,Rome
2,20040,NaN,Air France,Oslo,Newyork
3,20050,43 5,Swiss Air,Stockholm,Berlin


# Fixing the `Delays` column

We want to create (add) separate columns for the delays.

In [26]:
df['Delays']

0    20 4 1
1        13
2       NaN
3      43 5
Name: Delays, dtype: object

In [27]:
df['Delays'].str.split()

0    [20, 4, 1]
1          [13]
2           NaN
3       [43, 5]
Name: Delays, dtype: object

In [28]:
d = df['Delays'].str.split().apply(pd.Series)
d

,0,1,2
0,20,4,1
1,13,NaN,NaN
2,NaN,NaN,NaN
3,43,5,NaN


In [29]:
d.columns = ['delay 1', 'delay 2', 'delay 3']

d.columns = [ 'delay {}'.format(x) for x in range(1, len(d.columns)+1)]
d.columns

Index(['delay 1', 'delay 2', 'delay 3'], dtype='object')

In [30]:
d

,delay 1,delay 2,delay 3
0,20,4,1
1,13,NaN,NaN
2,NaN,NaN,NaN
3,43,5,NaN


Suppose we know 'NaN' delay means no delay (delay=0), we can replace the NaN by 0:

In [31]:
d = d.fillna(0)
d

,delay 1,delay 2,delay 3
0,20,4,1
1,13,0,0
2,0,0,0
3,43,5,0


In [32]:
df = df.join(d)
df

,SerialNumber,Delays,Airline,From,To,delay 1,delay 2,delay 3
0,20020,20 4 1,United Airlines,Paris,London,20,4,1
1,20030,13,British Airways,Madrid,Rome,13,0,0
2,20040,NaN,Air France,Oslo,Newyork,0,0,0
3,20050,43 5,Swiss Air,Stockholm,Berlin,43,5,0


now we can drop the original "Delays" column:

In [33]:
df = df.drop('Delays', axis=1)
df

,SerialNumber,Airline,From,To,delay 1,delay 2,delay 3
0,20020,United Airlines,Paris,London,20,4,1
1,20030,British Airways,Madrid,Rome,13,0,0
2,20040,Air France,Oslo,Newyork,0,0,0
3,20050,Swiss Air,Stockholm,Berlin,43,5,0


# Output the final results

Write the output to a csv file. We can reorganize the columns during writing. Set 'index' to false so row indices are not included in the csv.

In [34]:
df.to_csv('flight_new.txt', index=False)